In [1]:
from urllib.request import urlopen
import json
import pandas as pd
from flatten_json import flatten

In [2]:
#read Lake ID List
lakeid=pd.read_csv('/Users/kevinsoderholm/Desktop/PythonProjects/BWCA_APPATHON/CookLakeMapping.csv',dtype=str)

In [3]:

idlist=list(lakeid['ID'].unique())
#'38010400',
#idlist=['16013900','38070100','69047000']
#'38070100','69047000']
#,
idlist

['16018400',
 '16051500',
 '16035900',
 '16011400',
 '16032000',
 '16031100',
 '16075900',
 '16062200',
 '16061800',
 '16046900',
 '16047000',
 '16020400',
 '16008800',
 '16040700',
 '16025700',
 '16048600',
 '16018200',
 '16035000',
 '16035800',
 '16070100',
 '16075200',
 '16016400',
 '16007200',
 '16020500',
 '16022800',
 '16006700',
 '16030000',
 '16006300',
 '16005200',
 '16031200',
 '16065900',
 '16064700',
 '16034400',
 '16009800',
 '16062700',
 '16024700',
 '16053200',
 '16015100',
 '16022802',
 '16005000',
 '16038300',
 '16021100',
 '16017500',
 '16004400',
 '16046500',
 '16060000',
 '16034800',
 '16036400',
 '16027000',
 '16047700',
 '16027400',
 '16039700',
 '16011200',
 '16014100',
 '16024000',
 '16036000',
 '16022100',
 '16007100',
 '16034600',
 '16043800',
 '16042500',
 '16064400',
 '16007400',
 '16067500',
 '16052400',
 '16003300',
 '16037300',
 '16059400',
 '16011000',
 '16051800',
 '16036500',
 '16013900',
 '16044600',
 '16058100',
 '16053800',
 '16027100',
 '16035700',

In [4]:
def fishdata(acr):
    path='result_surveys_'+maxsurvey+'_lengths_'+acr+'_fishCount_'
    cols=[col for col in djc.columns if col.startswith(path)]
    data=djc[cols]
    lencols=[col for col in cols if col.endswith('0')]
    dj1=[]
    dj2=[]
    cols2=[]
    for i in range(0,len(lencols)):
        dj1.append(data[path+'{}'.format(i)+'_0'][0])
        cols2.append(acr+'Len_'+'{}'.format(dj1[i]))
        dj2.append(data[path+'{}'.format(i)+'_1'][0])
    df=pd.DataFrame([dj2],columns=cols2)
    return df
fishlengths=pd.DataFrame()
for i in idlist:
    #Retrieve data from URL in JSON format
    url="https://maps2.dnr.state.mn.us/cgi-bin/lakefinder/detail.cgi?type=lake_survey&id="+i
    response=urlopen(url)
    dja=json.loads(response.read())

    #flatten JSON and put in DataFrame
    djb=flatten(dja)
    djc=pd.json_normalize(djb).reset_index()
    djc=djc.astype(str)
    if djc['status'][0]=='ERROR':
        continue
    
     #Identify the different surveys
    datecols=[col for col in djc.columns if 'surveyDate' in col]
    idcols=['result_DOWNumber','result_lakeName']

    numsurveys = len(datecols)
    #djsurveys=djc[datecols+idcols]

    #Extract survey years
    for y in range(0,numsurveys):
        djc['year'+'{}'.format(y)]=djc['result_surveys_'+'{}'.format(y)+'_surveyDate'].str.slice(0,4)
    yearcols=[col for col in djc.columns if 'year' in col]
    djyear=djc[yearcols]
    djyear[yearcols].astype(int)

    #Identify max (most recent survey year)
    for z in range(0,numsurveys):
        djyear['year'+'{}'.format(z)]=pd.to_numeric(djyear['year'+'{}'.format(z)])
    maxsurvey=djyear[yearcols].idxmax(axis=1)
    maxsurvey=maxsurvey[0]
    maxsurvey=maxsurvey[4:]

    SMB=fishdata('SMB')
    SMB = SMB.loc[:,~SMB.columns.duplicated()] 
    NOP=fishdata('NOP')
    NOP = NOP.loc[:,~NOP.columns.duplicated()] 
    WAE=fishdata('WAE')
    WAE = WAE.loc[:,~WAE.columns.duplicated()] 
    LAT=fishdata('LAT')
    LAT = LAT.loc[:,~LAT.columns.duplicated()] 
    lengths=pd.concat([SMB,NOP,WAE,LAT],axis=1)
    if not lengths.empty:
        lengths['LakeID']='{}'.format(i)
        fishlengths=pd.concat([fishlengths,lengths], axis=0,ignore_index=False, sort=False)

/var/folders/l4/kn807d4s6kl5z1bvt1r084440000gn/T/ipykernel_60145/1809769126.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  djyear['year'+'{}'.format(z)]=pd.to_numeric(djyear['year'+'{}'.format(z)])
/var/folders/l4/kn807d4s6kl5z1bvt1r084440000gn/T/ipykernel_60145/1809769126.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  djyear['year'+'{}'.format(z)]=pd.to_numeric(djyear['year'+'{}'.format(z)])
/var/folders/l4/kn807d4s6kl5z1bvt1r084440000gn/T/ipykernel_60145/1809769126.py:45: SettingWithCopyWarnin

In [5]:
fishlengths

,SMBLen_14,SMBLen_15,NOPLen_29,NOPLen_30,WAELen_13,WAELen_14,WAELen_15,WAELen_16,WAELen_17,WAELen_19,...,NOPLen_1,LATLen_33,LATLen_40,WAELen_30,SMBLen_21,WAELen_31,NOPLen_42,LATLen_4,LATLen_5,WAELen_5
0,1,1,1,1,1,5,3,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,NaN,NaN,NaN,1,1,4,3,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6,2,NaN,NaN,3,3,6,6,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,2,1,NaN,NaN,5,5,2,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,10,11,2,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
0,NaN,NaN,NaN,NaN,1,4,NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
lengths

""
0


In [6]:
fishlengths.to_csv('/Users/kevinsoderholm/Desktop/PythonProjects/BWCA_APPATHON/FishSurveyCook.csv',index=False)

In [17]:
lake=pd.read_csv('/Users/kevinsoderholm/Desktop/PythonProjects/BWCA_APPATHON/FishSurveyLake.csv')
stlouis=pd.read_csv('/Users/kevinsoderholm/Desktop/PythonProjects/BWCA_APPATHON/FishSurveyStlouis.csv')
cook=pd.read_csv('/Users/kevinsoderholm/Desktop/PythonProjects/BWCA_APPATHON/FishSurveyCook.csv')
FishSurveyFull=pd.concat([lake,stlouis,cook], axis=0,ignore_index=False, sort=False)
FishSurveyFull

,SMBLen_14,SMBLen_15,SMBLen_16,NOPLen_19,NOPLen_20,NOPLen_22,NOPLen_24,NOPLen_25,NOPLen_26,NOPLen_27,...,WAELen_1,LATLen_6,LATLen_30,LATLen_34,LATLen_39,NOPLen_1,LATLen_33,LATLen_40,SMBLen_21,LATLen_4
0,1.0,2.0,4.0,1.0,2.0,1.0,1.0,3.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,3.0,2.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,3.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,3.0,1.0,2.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,NaN,NaN,NaN,3.0,4.0,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
FishSurveySum=FishSurveyFull.sum().to_frame()
FishSurveySum.reset_index(inplace=True)
FishSurveySum


,index,0
0,SMBLen_14,367.0
1,SMBLen_15,324.0
2,SMBLen_16,262.0
3,NOPLen_19,1509.0
4,NOPLen_20,1566.0
...,...,...
130,NOPLen_1,1.0
131,LATLen_33,1.0
132,LATLen_40,1.0
133,SMBLen_21,1.0


FishSurveyFull

In [12]:
#FishSurveySum.to_csv('/Users/kevinsoderholm/Desktop/PythonProjects/BWCA_APPATHON/FishSurveySum.csv')

In [22]:
fss=pd.read_csv('/Users/kevinsoderholm/Desktop/PythonProjects/BWCA_APPATHON/FishSurveySum.csv')
fss

,Length,LATcnt,LATpct,NOPcnt,NOPpct,SMBcnt,SMBpct,WAEcnt,WAEpct
0,0,0,0.0000,0,0.0000,37,0.0075,0,0.0000
1,1,0,0.0000,1,0.0001,660,0.1337,46,0.0025
2,2,0,0.0000,41,0.0025,277,0.0561,206,0.0113
3,3,0,0.0000,45,0.0028,70,0.0142,161,0.0088
4,4,1,0.0005,30,0.0018,84,0.0170,120,0.0066
5,5,4,0.0022,23,0.0014,302,0.0612,436,0.0239
6,6,4,0.0022,10,0.0006,250,0.0506,285,0.0156
7,7,32,0.0176,11,0.0007,277,0.0561,756,0.0415
8,8,48,0.0263,48,0.0029,253,0.0512,575,0.0315
9,9,53,0.0291,106,0.0065,253,0.0512,1095,0.0600


""
0
